<a href="https://colab.research.google.com/github/scarioscia/modeling_biological_populations/blob/main/Day_4_Advanced_Wright_Fisher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# **Live Coding**

In [ ]:
def generate_genotype(AF):
  if random.uniform(0, 1) < AF:
    return 'A'
  else: 
    return 'a'

def initialize_population(AF, N):
  pop = []
  for i in range(N):
    pop.append([generate_genotype(AF), generate_genotype(AF)])
  return pop

def make_mating_pair(pop):
  parents_are_different = False
  while parents_are_different == False:
    parents = random.choices(range(len(pop)) ,k =2)
    if parents[0] != parents [1]:
      parents_are_different = True
  return([pop[parents[0]], pop[parents[1]]])

def random_mating(pop, N):
  pop_next_gen = []
  for i in range(N):
    parents = make_mating_pair(pop)
    offspring = []
    for parent in parents: 
      offspring.append(random.choice(parent))
    pop_next_gen.append(offspring)
  return pop_next_gen

def get_AF(pop):
  A_counter = 0
  for individual in pop:
    for allele in individual:
      if allele == 'A':
        A_counter = A_counter + 1
  return A_counter / (2 * len(pop))

In [ ]:
def get_weights(pop, s_AA, s_Aa, s_aa):
  weights = []
  for individual in pop:
    if individual[0] == "A" and individual[0] == "A":
      weights.append(s_AA)
    elif individual[0] == "a" and individual[0] == "a":
      weights.append(s_aa)
    else:
      weights.append(s_Aa)
  return weights

def make_mating_pair_weighted(pop, s_AA, s_Aa, s_aa):
  parents_are_different = False
  weights = get_weights(pop, s_AA, s_Aa, s_aa)
  while parents_are_different == False:
    parents = random.choices(range(len(pop)), weights = weights,k =2)
    if parents[0] != parents [1]:
      parents_are_different = True
  return([pop[parents[0]], pop[parents[1]]])

def random_mating_weighted(pop, N, s_AA, s_Aa, s_aa):
  pop_next_gen = []
  for i in range(N):
    parents = make_mating_pair_weighted(pop, s_AA, s_Aa, s_aa)
    offspring = []
    for parent in parents: 
      offspring.append(random.choice(parent))
    pop_next_gen.append(offspring)
  return pop_next_gen

def WF_selection(AF, N, s_AA, s_Aa, s_aa, n_gens = 100, use_n_gens = False):
  # Initialize a population
  pop = initialize_population(AF, N)

  AF_list = []
  AF = get_AF(pop)
  AF_list.append(AF)

  if use_n_gens == True:
    for i in range(n_gens):
      pop = random_mating_weighted(pop, N, s_AA, s_Aa, s_aa)
      AF = get_AF(pop)
      AF_list.append(AF)
  else:
    while AF < 1 and AF > 0: 
      # Update population
      pop = random_mating_weighted(pop, N, s_AA, s_Aa, s_aa)
      AF = get_AF(pop)
      AF_list.append(AF)

  return AF_list

def WF(AF, N, n_gens = 100, use_n_gens = False):
  # Initialize a population
  pop = initialize_population(AF, N)

  AF_list = []
  AF = get_AF(pop)
  AF_list.append(AF)

  if use_n_gens == True:
    for i in range(n_gens):
      pop = random_mating(pop, N)
      AF = get_AF(pop)
      AF_list.append(AF)
  else:
    while AF < 1 and AF > 0: 
      # Update population
      pop = random_mating(pop, N)
      AF = get_AF(pop)
      AF_list.append(AF)

  return AF_list

In [ ]:
AFs = []
AFs2 = []
time_to_fixation = []
time_to_fixation2 = []
n_trials = 10
for i in range(n_trials):
  AF = WF(0.05, 100, use_n_gens = True )
  AF2 = WF_selection(AF = 0.05, N = 100, s_AA = 1.03, s_Aa = 1.03, s_aa=1, n_gens = 200, use_n_gens = True)
  AFs.append(AF)
  time_to_fixation.append(len(AF))
  AFs2.append(AF2)
  time_to_fixation2.append(len(AF))

def get_avg_line(af_list, time):
  longest_time = max(time)
  for i in range(len(af_list)):
    af_list[i] = af_list[i] + [af_list[i][-1]] * (longest_time - len(af_list[i]))
  af_list = np.array(af_list, dtype = 'float')
  avg_line = np.nanmean(af_list, axis = 0)
  return(avg_line)

fig ,axs = plt.subplots(2, figsize = (15, 6))
for i in range(n_trials):
  axs[0].plot(AFs[i], c = 'k')
  axs[1].plot(AFs2[i], c ='k')
axs[0].plot(get_avg_line(AFs, time_to_fixation), c = 'r')
axs[1].plot(get_avg_line(AFs2, time_to_fixation2), c = 'r')
plt.xlabel('Generation')
plt.ylabel('Frequency')
plt.show()

#**Basic Assignment: Bottlenecks**

During live coding, we generated a Wright-Fisher simulation and we modified it to introduce selection. 

Now let's look at the effect of population size. A **population bottleneck** occurs whenever the population size is reduced dramatically. This happens very often throughout a population's history – environmental disasters are a common source. But also, a population bottleneck often happens during migration – when a smaller group from a population leaves to form a new population, this creates a bottleneck (bottleneck due to migration is called the **founder effect**). 

Add in a small chance for a bottleneck to happen at any given generation and plot allele frequency trajectories over time. Also create a plot that keeps track of population size. 

#**Advanced Exercise: Bottlenecks and Logistic Growth**

After a bottleneck, the population grows back to its carrying capacity, *K*. Modify your functions so that at every generation, your population size updates based on the relationship between the current population size *N* and the carrying capacity *K*. You will likely want *K* be your starting population size. You will also need a growth rate, *r* – have this be a user-defined parameter. 

This model should have a population where the size crashes and then recovers over time. 